# CNN tests

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,random_split

from classiferNN import CNN,train_model,evaluate_model
from utils import compute_mean_std,show_images
%load_ext autoreload
%autoreload 2

* Set Up

In [2]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed=42
set_seed(seed)

In [3]:
device = 'cpu' #torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_workers=0
pin_memory=False
print(f"Device used: {device}")

Device used: cpu


In [4]:
mean= 0.3240 
std= 0.3612
batch_size = 128
transform = transforms.Compose([
    transforms.ToTensor(),            # Convert to tensor
    transforms.CenterCrop(26),
    transforms.Normalize(mean=(mean,),std=(std,)),
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

g = torch.Generator()
g.manual_seed(seed)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers, pin_memory=pin_memory)

In [5]:
# Split into train and validation (80/20)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size], generator=g)

# DataLoaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=pin_memory)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False,num_workers=num_workers, pin_memory=pin_memory)

In [24]:
model = CNN(dim = 26,conv_channels= [4,4,4],fc_dims=[16,16],kernel_size =4,stride=3,padding=2,max_pooling=False)
model

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 4, kernel_size=(4, 4), stride=(3, 3), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(4, 4, kernel_size=(4, 4), stride=(3, 3), padding=(2, 2))
    (3): ReLU()
    (4): Conv2d(4, 4, kernel_size=(4, 4), stride=(3, 3), padding=(2, 2))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=10, bias=True)
  )
)

In [25]:
model.get_num_params()

Total trainable parameters: 1302


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs=20

train_model(model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            optimizer=optimizer,
            criterion=criterion,
            num_epochs=epochs,
            device=device)

Epoch 1/20 completed in 12.38s.
Epoch 2/20 completed in 12.02s.
Epoch 3/20 completed in 13.16s.
Epoch 4/20 completed in 13.12s.
Epoch 5/20 completed in 14.01s.
Validation Accuracy after epoch 5: 80.02%
Epoch 6/20 completed in 13.50s.
Epoch 7/20 completed in 12.67s.
Epoch 8/20 completed in 12.94s.
Epoch 9/20 completed in 12.82s.
Epoch 10/20 completed in 13.03s.
Validation Accuracy after epoch 10: 82.41%
Epoch 11/20 completed in 12.13s.
Epoch 12/20 completed in 12.82s.
Epoch 13/20 completed in 12.57s.
Epoch 14/20 completed in 12.95s.
Epoch 15/20 completed in 12.93s.
Validation Accuracy after epoch 15: 82.95%
Epoch 16/20 completed in 12.22s.
Epoch 17/20 completed in 12.07s.
Epoch 18/20 completed in 12.92s.
Epoch 19/20 completed in 12.88s.
Epoch 20/20 completed in 12.49s.
Validation Accuracy after epoch 20: 83.05%


In [27]:
evaluate_model(model=model,data_loader=test_loader,device=device)

83.0